<center> <img src="logo_ifba.jpg" alt="jpg_python" width="100" height=""> </center>
<br><br>
<div align="center"><span style="font-size: 26px;"><strong>Análise de Dados Com Python</strong></span>
</div><br><br></center>
<center> <img src="python_gif.gif" alt="gif_python" width="80"> </center>

<div style="border: 2px solid black; padding: 10px; width: 100%; background-color: lightgray; display: flex; align-items: center;">
    <h1 style="color: red; text-align: center; margin: auto;">
        Exemplo Básico Envolvendo Mistura Gaussiana
    </h1>
</div>

<font color='red'></font>
<a href=""></a>

Vamos criar um exemplo básico de mistura Gaussiana que se pode resolver à mão, ideal para compreensão. Este exemplo será simplificado para facilitar os cálculos manuais, usando apenas duas distribuições Gaussianas (normais) e um conjunto pequeno de dados.

In [ ]:
import numpy as np
from scipy.stats import norm

# Exemplo: Mistura de Duas Gaussianas

Imagine que temos um conjunto pequeno de dados representando alturas (em centímetros) de uma população de adultos, e suspeitamos que esses dados vêm de uma mistura de duas populações distintas (por exemplo, homens e mulheres).

Conjunto de Dados: $160, 165, 170, 175, 180, 185$.

Nossa tarefa é modelar esses dados usando uma mistura de duas Gaussianas. Para simplificar, vamos supor algumas coisas:

1. **Médias Iniciais ($\mu_{1}$ e $\mu_{2}$)**: Vamos começar supondo que uma Gaussiana tem média $165$ (a mais baixa) e a outra $180$ (a mais alta).

2. **Variâncias Inciais ($\sigma_{1}^{2}$ e $\sigma_{2}^{2}$)**: Ambas as Gaussianas tem variância $25$.

3. **Pesos Inciais ($\pi_{1}$ e $\pi_{2}$)**: Cada Gaussiana contribui igualmente para o modelo, então ambos pesos são $0.5$.

*Entenda $\mu_{1}, \mu_{2}, \sigma_{1}, \sigma_{2}, \pi_{1}$ e $\pi_{2}$ dados como $\mu_{1}^{(0)}, \mu_{2}^{(0)}, \sigma_{1}^{(0)}, \sigma_{2}^{(0)}, \pi_{1}^{(0)}$ e $\pi_{2}^{(0)}$ uma vez que a mistura Gaussiana se trata de um método iterativo. Omitiremos as iterações $^{(n)}$ por questões de simplificação.*

In [ ]:
# Dados de exemplo
dados = np.array([160, 165, 170, 175, 180, 185])

# Parâmetros iniciais
mu = np.array([165, 180]) # mu1 e mu2
sigma = np.array([5, 5]) # sigma1 e sigma2
pi = np.array([0.5, 0.5]) # peso1 e peso2
n = len(dados)

## **Passo 1**: Calcular a probabilidade de cada ponto de dado para cada Gaussiana

Usamos a fórmula de densidade de probabilidade para uma distribuição normal:

$$P(x|\mu,\sigma^{2}) = \frac{1}{\sqrt{2\pi\sigma^{2}}}e^{-\frac{(x - \mu)^{2}}{2\sigma^{2}}}$$

Para cada ponto dado $x$, calculamos sua probabilidade sob as duas Gaussianas. Por exemplo, para $x = 160$ (dados[0]):

$$P(160|\mu = 165, \sigma^{2} = 25) \ e \ P(160|\mu = 180, \sigma^{2} = 25)$$

In [ ]:
print(f'{norm.pdf(dados[0], mu[0], sigma[0])} e {norm.pdf(dados[1], mu[1], sigma[1])}')

## **Passo 2**: Calcular a responsabilidade de cada Gaussiana para cada ponto dado

A responsabilidade $(\gamma)$ de cada Gaussiana é calculada usando a fórmula:

$$\gamma(z_{nk}) = \frac{\pi_{k}P(x_{n}|\mu_{k},\sigma_{k}^{2})}{\sum_{j=1}^{k} \pi_{j}P(x_{n}|\mu_{j},\sigma_{j}^{2})}$$

onde

$\bullet \gamma(z_{nk})$ é a probabilidade posterior de que o ponto $n$ seja gerado pela Gaussiana $k$.

$\bullet \pi_{k}$ é o peso da $k$-ésima Gaussiana.

$\bullet P(x_{n}|\mu_{k},\sigma_{k}^{2})$ é a probabilidade de $x_{n}$ sob a $k$-ésima Gaussiana.

O denominador é a soma das probabilidades ponderadas de $x_{n}$ sob todas as Gaussianas. Por exemplo, 

Para o primeiro ponto de dados ($160 = dados[0]$) e Gaussiana $1$ ($\mu_{1} = 165, \sigma_{1} = 5$):

$$\gamma_{11} = \frac{0.5 P(\boxed{160}|165,25)}{0.5P(\boxed{160}|165,25)+0.5P(\boxed{160}|180,25)} = 0.9994472213630764$$

Veja na célula a seguir como obter $\gamma_{11}$ no Jupyter:

In [ ]:
gamma11 = pi[0]*norm.pdf(dados[0], mu[0], sigma[0])/(pi[0]*norm.pdf(dados[0], mu[0], sigma[0]) + pi[1]*norm.pdf(dados[0], mu[1], sigma[1]))
print(gamma11)

De modo análogo obtem-se $\gamma_{12}$, $\gamma_{13}$, $\gamma_{14}$, $\gamma_{15}$, $\gamma_{21}$, $\gamma_{22}$, $\gamma_{23}$, $\gamma_{24}$ e $\gamma_{25}$. O código a seguir obtem todos os valores $\gamma(z_{nk})$:

In [ ]:
# Cálculo das responsabilidades
gamma = np.zeros((n, 2))

for i in range(n):
    denom = pi[0] * norm.pdf(dados[i], mu[0], sigma[0]) + pi[1] * norm.pdf(dados[i], mu[1], sigma[1])
    gamma[i, 0] = pi[0] * norm.pdf(dados[i], mu[0], sigma[0]) / denom
    gamma[i, 1] = pi[1] * norm.pdf(dados[i], mu[1], sigma[1]) / denom

print('     gamma_n1,       gamma_n2')
print(gamma)

## **Passo 3**: Atualizar os parâmetros das Gaussianas

Com base nas responsabilidades calculadas, atualizamos os pesos, médias e variância das Gaussianas:

$\bullet$ **Novos Pesos ($\pi_{k}$)**: A média das responsabilidades para cada Gaussiana.

$$\pi_{1}^{(n+1)} = \frac{\sum_{i=1}^{N} \gamma_{1i}}{N} \ e \ \pi_{1}^{(n+1)} = \frac{\sum_{i=1}^{N} \gamma_{2i}}{N}$$

$\bullet$ **Novas Médias ($\mu_{k}$)**: Uma média ponderada dos dados, onde os pesos são responsabilidades.

$$\mu_{1}^{(n+1)} = \frac{\sum_{i=1}^{N} \gamma_{1i}x_{i}}{\sum_{i=1}^{N} \gamma_{1i}} \ e \ \mu_{2}^{(n+1)} = \frac{\sum_{i=1}^{N} \gamma_{2i}x_{i}}{\sum_{i=1}^{N} \gamma_{2i}}$$

$\bullet$ **Novas Variâncias ($\sigma_{k}^{2}$)**: Uma média ponderada das diferenças quadradas dos dados em relação às novas médias, novamente ponderada pelas responsabilidades.

$$(\sigma^{2}_{1})^{(n+1)} = \frac{\sum_{i=1}^{N}(x_{i}-\mu_{1}^{(n+1)})^{2}}{\sum_{i=1}^{N} \gamma_{1i}} \ e \ (\sigma^{2}_{2})^{(n+1)} = \frac{\sum_{i=1}^{N}(x_{i}-\mu_{2}^{(n+1)})^{2}}{\sum_{i=1}^{N} \gamma_{2i}} $$

O código a seguir nos fornece os códigos referentes às fórmulas anteriores:

In [ ]:
# M-step: Atualiza os parâmetros
Nk = np.sum(gamma, axis=0) # denominador
for k in range(2):
    mu[k] = np.sum(gamma[:, k] * dados) / Nk[k]
    sigma[k] = np.sqrt(np.sum(gamma[:, k] * (dados - mu[k])**2) / Nk[k])
    pi[k] = Nk[k] / n

In [ ]:
print(mu)

In [ ]:
print(sigma)

In [ ]:
print(pi)

## **Passo 4**: Iterações

Repita os passos 2 e 3 até que os parâmetros das Gaussianas convirjam (ou seja, não mudem significativamente entre iterações).

## Algoritmo Completo (duas populações)

Segue o algoritmo que realiza os cálculos acima:

In [18]:
import numpy as np
from scipy.stats import norm

def mistura_gaussiana_em(dados, mu_inicial, sigma_inicial, pi_inicial, n_iteracoes):
    # Inicializa os parâmetros
    mu = mu_inicial
    sigma = sigma_inicial
    pi = pi_inicial
    n = len(dados)
    
    for _ in range(n_iteracoes):
        # E-step: Calcula as responsabilidades
        gamma = np.zeros((n, 2))
        for i in range(n):
            denom = pi[0] * norm.pdf(dados[i], mu[0], sigma[0]) + pi[1] * norm.pdf(dados[i], mu[1], sigma[1])
            gamma[i, 0] = pi[0] * norm.pdf(dados[i], mu[0], sigma[0]) / denom
            gamma[i, 1] = pi[1] * norm.pdf(dados[i], mu[1], sigma[1]) / denom
        
        # M-step: Atualiza os parâmetros
        Nk = np.sum(gamma, axis=0)
        for k in range(2):
            mu[k] = np.sum(gamma[:, k] * dados) / Nk[k]
            sigma[k] = np.sqrt(np.sum(gamma[:, k] * (dados - mu[k])**2) / Nk[k])
            pi[k] = Nk[k] / n
        print(mu, sigma, pi)
    
    return mu, sigma, pi

# Dados de exemplo
dados = np.array([160, 165, 170, 175, 180, 185])

# Parâmetros iniciais
mu_inicial = np.array([165, 180])
sigma_inicial = np.array([5, 5])
pi_inicial = np.array([0.5, 0.5])

# Número de iterações
n_iteracoes = 5

# Executa o algoritmo
mu_final, sigma_final, pi_final = mistura_gaussiana_em(dados, mu_inicial, sigma_inicial, pi_inicial, n_iteracoes)

[165 179] [4 4] [0.5 0.5]
[164 179] [4 4] [0.48636863 0.51363137]
[164 179] [4 4] [0.47111729 0.52888271]
[164 179] [4 4] [0.46906663 0.53093337]
[164 179] [4 4] [0.46878716 0.53121284]


## Algoritmo Completo (k populações)

Apresentamos a seguir a versão do nosso código considerando a mistura Gaussiana com $m$ populações:

In [19]:
import numpy as np
from scipy.stats import norm

def mistura_gaussiana_em(dados, mu_inicial, sigma_inicial, n_iteracoes, K):
    n = len(dados)  # Número de pontos de dados

    # Inicializa os parâmetros
    mu = np.array(mu_inicial)
    sigma = np.array(sigma_inicial)
    pi = np.full(K, 1/K)  # Pesos iniciais iguais para todas as Gaussianas

    for _ in range(n_iteracoes):
        # E-step: Calcula as responsabilidades
        gamma = np.zeros((n, K))
        for i in range(n):
            denom = sum(pi[k] * norm.pdf(dados[i], mu[k], sigma[k]) for k in range(K))
            for k in range(K):
                gamma[i, k] = pi[k] * norm.pdf(dados[i], mu[k], sigma[k]) / denom

        # M-step: Atualiza os parâmetros
        Nk = np.sum(gamma, axis=0)
        for k in range(K):
            mu[k] = np.sum(gamma[:, k] * dados) / Nk[k]
            sigma[k] = np.sqrt(np.sum(gamma[:, k] * (dados - mu[k])**2) / Nk[k])
            pi[k] = Nk[k] / n

        # Opcional: imprimir os parâmetros para acompanhar a convergência
        print(mu, sigma, pi)

    return mu, sigma, pi

# Exemplo de uso com K populações
K = 3  # Número de Gaussianas/populações
dados = np.array([160, 165, 170, 175, 180, 185])
mu_inicial = [165, 175, 185]  # Médias iniciais para K Gaussianas
sigma_inicial = [5, 5, 5]  # Desvios padrão iniciais para K Gaussianas
n_iteracoes = 5 # Número de iterações

# Executar o algoritmo EM
mu_final, sigma_final, pi_final = mistura_gaussiana_em(dados, mu_inicial, sigma_inicial, n_iteracoes, K)

print("Médias finais:", mu_final)
print("Desvios padrão finais:", sigma_final)
print("Pesos finais:", pi_final)


[164 174 182] [4 5 3] [0.41231408 0.33903881 0.24864711]
[163 174 182] [4 4 2] [0.3828353  0.34479409 0.27237061]
[163 173 182] [3 4 2] [0.36975709 0.34560582 0.28463709]
[162 172 182] [3 3 2] [0.33320939 0.36582974 0.30096087]
[162 172 182] [2 3 2] [0.32080153 0.35128856 0.32790991]
Médias finais: [162 172 182]
Desvios padrão finais: [2 3 2]
Pesos finais: [0.32080153 0.35128856 0.32790991]
